In [74]:
from dotenv import load_dotenv
import os

In [75]:
# Load variables from .env into environment
load_dotenv()

# Access them with os.environ
api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")
debug_mode = os.getenv("DEBUG") == "True"

paper = True
data_api_url = None

In [76]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest

import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

In [77]:
stock_historical_data_client = StockHistoricalDataClient(
    api_key, secret_key, url_override=data_api_url
)

symbol = "GOOGL"

# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
now = datetime.now(ZoneInfo("America/Chicago"))
req = StockBarsRequest(
    symbol_or_symbols=[symbol],
    timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),  # specify timeframe
    start=now
    - timedelta(
        weeks=52 * 4
    ),  # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    # limit=1000,  # specify limit
)

df_bars = stock_historical_data_client.get_stock_bars(req).df

# Calculate 5-period and 20-period Simple Moving Average (SMA)
df_bars["SMA_5"] = df_bars["close"].rolling(window=5).mean()
df_bars["SMA_20"] = df_bars["close"].rolling(window=20).mean()

# Exclude days where SMA values are NaN
df_bars = df_bars.dropna(subset=["SMA_5", "SMA_20"])

# Sort bars by symbol and timestamp
df_bars = df_bars.sort_index(level=["symbol", "timestamp"])

print(f"Number of rows after dropping NaNs: {len(df_bars)}")

Number of rows after dropping NaNs: 980


## Check Conditions for Trading Signals


### Closing > Opening


In [78]:
is_up = df_bars["close"] > df_bars["open"]
is_up.head(3)

symbol  timestamp                
GOOGL   2021-10-14 04:00:00+00:00     True
        2021-10-15 04:00:00+00:00    False
        2021-10-18 04:00:00+00:00     True
dtype: bool

Check previous day's closing price is greater than opening price.


In [79]:
prev_up = is_up.groupby(level="symbol").shift(1)
prev_up.head(3)

symbol  timestamp                
GOOGL   2021-10-14 04:00:00+00:00      NaN
        2021-10-15 04:00:00+00:00     True
        2021-10-18 04:00:00+00:00    False
dtype: object

### Open below SMA_5 AND close above SMA_5


In [80]:
cross_above_sma5 = (df_bars["open"] < df_bars["SMA_5"]) & (
    df_bars["close"] > df_bars["SMA_5"]
)
cross_above_sma5.head(3)

symbol  timestamp                
GOOGL   2021-10-14 04:00:00+00:00    False
        2021-10-15 04:00:00+00:00    False
        2021-10-18 04:00:00+00:00    False
dtype: bool

### Previous day's close below SMA_5


In [81]:
prev_close_below_sma5 = (
    (df_bars["close"] < df_bars["SMA_5"]).groupby(level="symbol").shift(1)
)

prev_close_below_sma5.head(3)

symbol  timestamp                
GOOGL   2021-10-14 04:00:00+00:00      NaN
        2021-10-15 04:00:00+00:00    False
        2021-10-18 04:00:00+00:00    False
dtype: object

### SMA_5 below SMA_20


In [82]:
sma5_below_sma20 = df_bars["SMA_5"] < df_bars["SMA_20"]
sma5_below_sma20.head(3)

symbol  timestamp                
GOOGL   2021-10-14 04:00:00+00:00    False
        2021-10-15 04:00:00+00:00    False
        2021-10-18 04:00:00+00:00    False
dtype: bool

### Current AND previous day close < SMA_20


In [83]:
close_below_sma20 = df_bars["close"] < df_bars["SMA_20"]
prev_close_below_20 = close_below_sma20.groupby(level="symbol").shift(1)
both_close_below_sma20 = close_below_sma20 & prev_close_below_20
both_close_below_sma20.head(3)

symbol  timestamp                
GOOGL   2021-10-14 04:00:00+00:00    False
        2021-10-15 04:00:00+00:00    False
        2021-10-18 04:00:00+00:00    False
dtype: bool

---

## Combine all conditions to mark potential buy signals


In [84]:
mask = (
    is_up
    & prev_up
    & cross_above_sma5
    & prev_close_below_sma5
    & sma5_below_sma20
    & both_close_below_sma20
)

df_bars["potential_buy_signal"] = mask
# df_bars_potential_buy_signals = df_bars[mask]
# df_bars_potential_buy_signals
df_bars[df_bars["potential_buy_signal"]]

open      high        low    close  \
symbol timestamp                                                           
GOOGL  2022-01-11 05:00:00+00:00  2760.140  2804.320  2733.8450  2794.72   
       2022-04-19 04:00:00+00:00  2553.830  2606.660  2539.9900  2600.18   
       2022-08-25 04:00:00+00:00   114.235   116.720   114.1100   116.65   
       2022-10-13 04:00:00+00:00    95.150    99.775    94.3836    99.06   
       2023-02-15 05:00:00+00:00    94.490    97.120    94.1500    96.94   
       2023-03-01 05:00:00+00:00    89.980    91.030    89.6700    90.36   
       2023-08-22 04:00:00+00:00   128.510   130.278   128.3200   129.08   
       2024-08-16 04:00:00+00:00   161.470   165.060   161.1300   162.96   
       2024-11-26 05:00:00+00:00   167.630   169.820   167.5800   169.12   
       2025-02-13 05:00:00+00:00   184.320   186.280   183.1400   186.14   
       2025-03-21 04:00:00+00:00   161.205   164.240   160.8901   163.99   

                                      volume  trade_count         vwap  \
symbol timestamp                                                         
GOOGL  2022-01-11 05:00:00+00:00   1552393.0     100620.0  2781.602733   
       2022-04-19 04:00:00+00:00   1504116.0     102796.0  2589.482555   
       2022-08-25 04:00:00+00:00  19294089.0     222759.0   115.847827   
       2022-10-13 04:00:00+00:00  37849768.0     400539.0    97.755636   
       2023-02-15 05:00:00+00:00  53831053.0     449020.0    96.085784   
       2023-03-01 05:00:00+00:00  34280313.0     286691.0    90.341092   
       2023-08-22 04:00:00+00:00  22071182.0     231121.0   129.237825   
       2024-08-16 04:00:00+00:00  24208647.0     360419.0   163.283676   
       2024-11-26 05:00:00+00:00  20486720.0     268773.0   168.935164   
       2025-02-13 05:00:00+00:00  21402523.0     289687.0   184.821842   
       2025-03-21 04:00:00+00:00  36625764.0     295271.0   162.975843   

                                     SMA_5     SMA_20  potential_buy_signal  
symbol timestamp                                                             
GOOGL  2022-01-11 05:00:00+00:00  2763.780  2867.5660                  True  
       2022-04-19 04:00:00+00:00  2568.096  2726.6400                  True  
       2022-08-25 04:00:00+00:00   115.130   117.6675                  True  
       2022-10-13 04:00:00+00:00    98.068    99.3715                  True  
       2023-02-15 05:00:00+00:00    95.162    98.7560                  True  
       2023-03-01 05:00:00+00:00    90.062    96.1140                  True  
       2023-08-22 04:00:00+00:00   128.706   129.8470                  True  
       2024-08-16 04:00:00+00:00   162.216   166.6205                  True  
       2024-11-26 05:00:00+00:00   169.028   173.9465                  True  
       2025-02-13 05:00:00+00:00   185.376   194.4175                  True  
       2025-03-21 04:00:00+00:00   163.128   168.2125                  True

---

## Calculate


In [88]:
print(df_bars.index.names)
df_bars.head(10)

['symbol', 'timestamp']


open       high       low    close  \
symbol timestamp                                                            
GOOGL  2021-10-14 04:00:00+00:00  2789.4761  2826.7450  2776.500  2823.02   
       2021-10-15 04:00:00+00:00  2832.2900  2834.3600  2815.100  2827.36   
       2021-10-18 04:00:00+00:00  2821.5300  2855.9200  2821.389  2855.56   
       2021-10-19 04:00:00+00:00  2867.7500  2873.2500  2852.000  2864.74   
       2021-10-20 04:00:00+00:00  2866.7600  2870.9218  2827.530  2835.38   
       2021-10-21 04:00:00+00:00  2835.3800  2843.1200  2810.000  2837.72   
       2021-10-22 04:00:00+00:00  2783.0000  2811.6550  2721.120  2751.33   
       2021-10-25 04:00:00+00:00  2751.0000  2760.0000  2708.480  2748.94   
       2021-10-26 04:00:00+00:00  2785.2700  2801.6588  2766.090  2786.17   
       2021-10-27 04:00:00+00:00  2788.1000  2973.0000  2788.100  2924.35   

                                     volume  trade_count         vwap  \
symbol timestamp                                                        
GOOGL  2021-10-14 04:00:00+00:00  1763914.0      80318.0  2813.891813   
       2021-10-15 04:00:00+00:00  1749621.0      73292.0  2826.739808   
       2021-10-18 04:00:00+00:00  1157869.0      69315.0  2846.922126   
       2021-10-19 04:00:00+00:00  1135848.0      65856.0  2862.847809   
       2021-10-20 04:00:00+00:00  1250582.0      75991.0  2846.801453   
       2021-10-21 04:00:00+00:00  1408598.0      83576.0  2831.294159   
       2021-10-22 04:00:00+00:00  2755581.0     152794.0  2755.002750   
       2021-10-25 04:00:00+00:00  1824771.0     107000.0  2742.848981   
       2021-10-26 04:00:00+00:00  2600438.0     146505.0  2783.211643   
       2021-10-27 04:00:00+00:00  4552112.0     239402.0  2907.390882   

                                     SMA_5     SMA_20  potential_buy_signal  
symbol timestamp                                                             
GOOGL  2021-10-14 04:00:00+00:00  2775.526  2764.0955                 False  
       2021-10-15 04:00:00+00:00  2781.856  2764.6635                 False  
       2021-10-18 04:00:00+00:00  2797.312  2768.7220                 False  
       2021-10-19 04:00:00+00:00  2824.464  2772.9260                 False  
       2021-10-20 04:00:00+00:00  2841.212  2774.4115                 False  
       2021-10-21 04:00:00+00:00  2844.152  2775.0815                 False  
       2021-10-22 04:00:00+00:00  2828.946  2770.4330                 False  
       2021-10-25 04:00:00+00:00  2807.622  2766.8080                 False  
       2021-10-26 04:00:00+00:00  2791.908  2770.2865                 False  
       2021-10-27 04:00:00+00:00  2809.702  2782.1505                 False